In [2]:
import numpy as np
import sputter as sp
import matplotlib.pyplot as pl
import everest as ev
import csv
from numpy import genfromtxt
from everest import detrender
from everest.math import SavGol, Scatter, Downbin
from astropy.stats import median_absolute_deviation as mad
from numpy.polynomial import polynomial as P
from tqdm import tqdm
from scipy.optimize import curve_fit
% matplotlib qt
# % matplotlib inline

In [3]:
datapath = '/Users/nks1994/Documents/Research/everest/docs/c0'
dataloc = '.csv'

# CAMPAIGN 1

tags = []
mags = []
with open((datapath+str(1)+dataloc),'r') as f:
    data = csv.reader(f)
    for i,row in enumerate(data):
        if i == 0:
            continue
        else:
            tags.append(row[0])
            mags.append(row[1])

In [4]:
cdpps = []
for t in tags:
    star = ev.Everest(int(t))
    cdpps.append(star.cdpp)

INFO  [everest.user.DownloadFile()]: Found cached file.
INFO  [everest.user.load_fits()]: Loading FITS file for 201735103.
INFO  [everest.user.DownloadFile()]: Found cached file.
INFO  [everest.user.load_fits()]: Loading FITS file for 201505924.
INFO  [everest.user.DownloadFile()]: Found cached file.
INFO  [everest.user.load_fits()]: Loading FITS file for 201266988.


KeyboardInterrupt: 

In [5]:
cdpps = np.load('k21cdpps.npy')
length = len(cdpps)

In [6]:
fig = pl.figure(figsize=(15,5))
pl.plot(mags[:length],cdpps,'b.',alpha = 0.25)
pl.ylim([0,700])
pl.xlim([5,20])

(5, 20)

In [7]:
#def rawCDPP(mag,motion):
mag = 10
motion = 0
star = np.load('stars/larger_aperture/mag'+str(mag)+'motion'+str(motion)+'.npz')
fpix = star['fpix']


In [8]:
def simCDPP(mag,motion):

    MN = sp.MotionNoise()
    flux,rawflux = MN.DetrendFpix(mag,motion)

    CDPP = MN.CDPP(rawflux)

    return CDPP

In [9]:
rawcdpps = []
for i in tqdm(range(5)):
    n = i+10
    rawcdpps.append(simCDPP(n,1))

  0%|          | 0/5 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
simcdpps = []
for i in tqdm(range(5)):
    n = i + 10
    simcdpps.append(simCDPP(n,1))

In [ ]:
fig = pl.figure(figsize=(15,5))
pl.plot(mags[:length],cdpps,'b.',alpha = 0.25)
pl.ylim([0,6000])
pl.xlim([5,20])
xs = [i+10 for i in range(5)]
pl.plot(xs,rawcdpps,'g.',markersize=12)
pl.plot(xs,simcdpps,'r.',markersize=12)
pl.xlabel(r'$K_p\ Mag$')
pl.ylabel(r'$CDPP$')

In [13]:
t = np.linspace(0,20,1399)

from scipy.optimize import curve_fit

mags = [float(i) for i in mags] 

def fitFunc(x, a, b, c):
    return a * np.exp(b*x) + c

In [ ]:
fitParams, fitCovariance = curve_fit(fitFunc, t, cdpps, p0=(1,1e-5,1))

In [ ]:
fig = pl.figure(figsize=(15,5))
sigma = [fitCovariance[0,0], \
         fitCovariance[1,1], \
         ]
pl.plot(t, fitFunc(t, fitParams[0], fitParams[1], fitParams[2]))
#        t, fitFunc(t, fitParams[0] + sigma[0], fitParams[1] - sigma[1], fitParams[2] + sigma[2]),\
#        t, fitFunc(t, fitParams[0] - sigma[0], fitParams[1] + sigma[1], fitParams[2] - sigma[2]))

In [ ]:
pl.plot(mags[:length],cdpps,'k.')

In [ ]:
msort = sorted(mags,key=int)
d = {k:m for k,m in enumerate(msort[:length])}
csort = cdpps.sorted(key=d.get)

In [ ]:
csort = cdpps.reshape(mags[:length])

In [ ]:
motion = 0
mag = 14
path = 'stars/larger_aperture/mag' + str(mag) + 'motion' + str(motion) + '.npz'
fpix = np.load(path)['fpix']

fpix_rs = fpix.reshape(len(fpix),-1)
tempflux = np.sum(fpix_rs,axis=1)

In [ ]:
pl.plot(tempflux,'k.')

In [ ]:
import everest

In [ ]:
everest.missions.k2.CDPP(tempflux[tempflux>1000000])

In [ ]:
MN = sp.MotionNoise()
MN.CDPP(tempflux)

In [ ]:
star = everest.Everest(205071984)

In [ ]:
star.cdpp

In [ ]:
star.mag

In [ ]:
star.cdppr

In [ ]:
counts = []
N = []
for t in tags[:10]:
    t = int(t)
    star = everest.Everest(t)
    counts.append(np.mean(star.apply_mask(star.flux)))
    N.append((np.std(star.apply_mask(star.flux))/np.sqrt(np.mean(star.apply_mask(star.flux)))))

In [10]:
Afile = np.load('A.npz')
A = Afile['A']

Aguess = [1e7,1,0,-8]

In [14]:

def func(x, a, b, c, x0):
    return a * np.exp(-b * (x+x0)) + c

# Simulate data
counts = np.random.random(1000) * 200000
N = func(counts, 0.1, 1. / 2500, 0.04, 0) + 0.0025 * np.random.randn(1000)

# Plot data
pl.plot(counts, N, 'r.', alpha = 0.3, ms = 2)

# Fit the exponential
guess = [0.1, 1. / 2500, 0.04, 0]
coeffs, _ = curve_fit(func, counts, N, p0 = guess)

# Plot the fit
xfit = np.linspace(0, 200000, 1000)
pl.plot(xfit, func(xfit, *coeffs), 'k-')

# Show
pl.show() 

In [18]:
Acoeffs, _ = curve_fit(func, mags[:len(A)], A, p0 = Aguess)
Acoeffs

array([  1.64864230e+07,   9.12458354e-01,  -6.42400461e+03,
        -7.35240947e+00])

In [23]:
fig = pl.figure(figsize=(15,5))
x_ = np.linspace(0, mags[:len(A)][-1], 1000)
pl.plot(mags[:len(A)],A,'r.',alpha=0.5)
pl.plot(x_,func(x_, 1.64864230e+07,   .93,  -6.42400461e+03, -7.35240947e+00),'b')
#pl.xlim([8,20])
pl.ylim([0,1e7])
pl.show()

In [ ]:
array([  1.64864230e+07,   1.,  -6.42400461e+03, -7.35240947e+00])

In [ ]:
x_.shape

In [ ]:
A.shape

In [27]:
def amp(mag):
    x=mag
    a,b,c,d = 1.64864230e+07, .93, -7.35240947e+00, -6.42400461e+03
    return a * np.exp(-b * (x+c)) + d

In [30]:
amp(11)

548057.4049733557